In [54]:
%pylab
%matplotlib inline
%config InlineBackend.figure_format = 'svg'
import seaborn as sns
sns.set()

Using matplotlib backend: MacOSX
Populating the interactive namespace from numpy and matplotlib


/Users/marika/anaconda/lib/python2.7/site-packages/IPython/core/magics/pylab.py:161: UserWarning: pylab import has clobbered these variables: ['f', 'test']
`%matplotlib` prevents importing * from pylab and numpy
  "\n`%matplotlib` prevents importing * from pylab and numpy"


In [55]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import os.path
import scipy 
import scipy.stats as stats
import statsmodels as smr
import statsmodels.stats.api as sms
import statsmodels.api as sm
import statsmodels.formula.api as smf
scipy.stats.chisqprob = lambda chisq, df: scipy.stats.chi2.sf(chisq, df)

In [56]:
class Project:   
    def __init__(self):
        self.data = None
        self.train_data = None
        self.test_data = None
            
    def _load_data_(self,filepath):
      
        #if (~os.path.isfile('train.csv')) | (~os.path.isfile('test.csv')):
           # self.data = pd.read_csv(filepath,encoding='ISO-8859-1')
           # self.train_test_split()
           # self._write_data_()
            
        self.train_data = pd.read_csv('train.csv')
                                #,encoding='ISO-8859-1')
        self.test_data = pd.read_csv('test.csv',encoding='ISO-8859-1')
        return self.train_data, self.test_data
        
    def _write_data_(self):
        self.train_data.to_csv('train.csv')
        self.test_data.to_csv('test.csv')
    
    def train_test_split(self):
        self.train_data, self.test_data = train_test_split(self.data, test_size=0.2, random_state=42)   
        
    def null_summary(self,df):
        print(df.isnull().sum())
        all_data_na = (df.isnull().sum() / len(df)) * 100
        all_data_na = all_data_na.drop(all_data_na[all_data_na == 0].index).sort_values(ascending=False)
        missing_data = pd.DataFrame({'Missing Ratio' :all_data_na})
        missing_data.head()
        if missing_data.size> 0:
            f, ax = plt.subplots(figsize=(15, 12))
            plt.xticks(rotation='90')
            sns.barplot(x=all_data_na.index, y=all_data_na)
            plt.xlabel('Features', fontsize=15)
            plt.ylabel('Percent of missing values', fontsize=15)
            plt.title('Percent missing data by feature', fontsize=15)
            plt.show()
            
    def get_stats(self,df,variable):
        print("Skewness: %f" % df[variable].skew())
        print("Kurtosis: %f" % df[variable].kurt())
        print("Mean: %f" % df[variable].mean())
        print("Variance: %f" % (df[variable].var()))

            
    def correlation_map(self,df,outcome,k):
        corrmat = df.corr()
        col_large = corrmat.nlargest(k+1,outcome)[outcome].index
        col_small = corrmat.nsmallest(k,outcome)[outcome].index
        cols = col_large.union(col_small)
        cm = np.corrcoef(corrmat[cols].loc[cols].values.T)
        print(corrmat[cols].loc[outcome])
        #sns.set(font_scale=1.25)
        hm = sns.heatmap(cm, cbar=True, vmax=1,annot=True, fmt='.1f', annot_kws={'size': 8}, yticklabels=cols.values, xticklabels=cols.values)
        return cols
    
    def encode_organization(self,x):
            if 'Academy' in x:
                return 1
            elif 'Instituion' in x:
                return 2
            elif 'College' in x:
                return 3
            elif 'University' in x:
                return 4
            else:
                return 0
    
    def encode_cost(self,df):
        prices = []
        for i in range(len(df)):
            if pd.notnull(df['NPT4_PUB'][i]):
                prices.append(df['NPT4_PUB'][i])
            elif pd.notnull(df['NPT4_PRIV'][i]):
                prices.append(df['NPT4_PRIV'][i])
            else:
                prices.append(np.NaN) 
        
        df.loc[:,'NET_COST'] = pd.Series(prices, index=df.index)
        
        return df
    
    def encode_25KBinary(self, df):
        vals = []
        for i in range(len(df)):
            if float(df['gt_25k_p6'][i]) >= 0.6:
                vals.append(1)
            elif float(df['gt_25k_p6'][i]) < 0.6:
                vals.append(0)
            else:
                vals.append(np.NaN)
        
        df.loc[:,'Binary_25k'] = pd.Series(vals, index=df.index)
        
        return df
    
    def encode_state(self,x):
        west = ['WA','MT','OR','ID','WT','CA','NV','UT','CO','AZ','NM']
        midwest = ['ND','MN','SD','NE','KS','IA','MO','WI','IL','MI','IN','OH','WY']
        northeast = ['NY','PA','NJ','RI','CT','MA','VT','NH','ME']
        south = ['TX','OK','AR','LA','MS','AL','GA','TN','KY','WV','DC','MD','VA','DE','NC','SC','FL']
        others =['AK','HI','PR','PW','AS','GU','FM','VI']
        
        if x in others:
            return 0
        elif x in midwest:
            return 1
        elif x in south:
            return 2
        elif x in west:
            return 3
        elif x in northeast:
            return 4
        else: return x
        
    def compute_ci(self,x):
        print('tn,p,dof =',sms.DescrStatsW(x).ttest_mean(x.mean()))
        print('CI =',sms.DescrStatsW(x).tconfint_mean())

In [57]:
project = Project()
project._load_data_('test.csv')

(      Unnamed: 0                                             INSTNM  \
 0           7440       Marian Health Careers Center-Van Nuys Campus   
 1           5144                   Emma's Beauty Academy-Juana Diaz   
 2           7169                              Milan Institute-Nampa   
 3           5160              University of Phoenix-Bay Area Campus   
 4           4987                      Sanford-Brown Institute-Tampa   
 5           2415                  Truckee Meadows Community College   
 6           6076                             Healing Arts Institute   
 7           1790          Maryland University of Integrative Health   
 8           7170             Milan Institute of Cosmetology-El Paso   
 9           3857                  Middle Tennessee State University   
 10          3260           Virginia Marti College of Art and Design   
 11          3817                                Lipscomb University   
 12          1670                           McNeese State Univer

In [58]:
test_df = project.test_data
train_df = project.train_data
train_df = train_df.drop(train_df.columns[0],axis=1)
train_df.info()
test_df = test_df.drop(test_df.columns[0],axis=1)
test_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6243 entries, 0 to 6242
Data columns (total 42 columns):
INSTNM                    6243 non-null object
CITY                      6243 non-null object
STABBR                    6243 non-null object
PREDDEG                   6243 non-null int64
CONTROL                   6243 non-null int64
LOCALE                    5911 non-null float64
SATVRMID                  1036 non-null float64
SATMTMID                  1048 non-null float64
SATWRMID                  631 non-null float64
ACTCMMID                  1076 non-null float64
ACTENMID                  939 non-null float64
ACTMTMID                  940 non-null float64
ACTWRMID                  247 non-null float64
SAT_AVG                   1136 non-null float64
DISTANCEONLY              5913 non-null float64
UGDS                      5680 non-null float64
UGDS_WHITE                5680 non-null float64
UGDS_BLACK                5680 non-null float64
UGDS_HISP                 5680 non-null 

In [59]:
print (train_df.shape)
print (test_df.shape)

(6243, 42)
(1561, 42)


In [60]:
train_df = train_df.apply(lambda x: x.replace('PrivacySuppressed',np.NAN))
train_df = project.encode_cost(train_df)
train_df = project.encode_25KBinary(train_df)

In [61]:
test_df = test_df.apply(lambda x: x.replace('PrivacySuppressed',np.NAN))
test_df = project.encode_cost(test_df)
test_df = project.encode_25KBinary(test_df)

In [62]:
train_df['RPY_3YR_RT_SUPP'] = pd.to_numeric(train_df['RPY_3YR_RT_SUPP'])
train_df['md_earn_wne_p10'] = pd.to_numeric(train_df['md_earn_wne_p10'])
train_df['GRAD_DEBT_MDN_SUPP'] = pd.to_numeric(train_df['GRAD_DEBT_MDN_SUPP'])
train_df['GRAD_DEBT_MDN10YR_SUPP'] = pd.to_numeric(train_df['GRAD_DEBT_MDN10YR_SUPP'])
train_df['C150_4_POOLED_SUPP'] = pd.to_numeric(train_df['C150_4_POOLED_SUPP'])
train_df['C200_L4_POOLED_SUPP'] = pd.to_numeric(train_df['C200_L4_POOLED_SUPP'])

In [63]:
test_df['RPY_3YR_RT_SUPP'] = pd.to_numeric(test_df['RPY_3YR_RT_SUPP'])
test_df['md_earn_wne_p10'] = pd.to_numeric(test_df['md_earn_wne_p10'])
test_df['GRAD_DEBT_MDN_SUPP'] = pd.to_numeric(test_df['GRAD_DEBT_MDN_SUPP'])
test_df['GRAD_DEBT_MDN10YR_SUPP'] = pd.to_numeric(test_df['GRAD_DEBT_MDN10YR_SUPP'])
test_df['C150_4_POOLED_SUPP'] = pd.to_numeric(test_df['C150_4_POOLED_SUPP'])
test_df['C200_L4_POOLED_SUPP'] = pd.to_numeric(test_df['C200_L4_POOLED_SUPP'])

In [64]:
columns = train_df.columns
#drop outcomes variables and "name" variables 
treatment_variables = columns.drop(['INSTNM','CITY','STABBR','RPY_3YR_RT_SUPP', 'md_earn_wne_p10', 'gt_25k_p6', 'Binary_25k'])

In [65]:
columnst = test_df.columns
#drop outcomes variables and "name" variables 
treatment_variables = columnst.drop(['INSTNM','CITY','STABBR','RPY_3YR_RT_SUPP', 'md_earn_wne_p10', 'gt_25k_p6', 'Binary_25k'])

In [66]:
#only keep the variables if less than 20% of the values are missing ( SHOULD I KEEP IT at 20%?)
keep_variables = []
for i in range(len(treatment_variables)):
    x = train_df[str(treatment_variables[i])].isnull().sum()/float((len(train_df)))
    if x <= 0.2:
        keep_variables.append(treatment_variables[i]) 

In [67]:
drop_cols = []
for i in train_df.columns:
    if i =='RPY_3YR_RT_SUPP':
        next
    elif i == 'Binary_25k':
        next
    elif i not in keep_variables:
        drop_cols.append(i)
    elif i == 'GRAD_DEBT_MDN10YR_SUPP':
        drop_cols.append(i)
    elif i == 'UGDS_UNKN':
        drop_cols.append(i)

In [69]:
import scipy.special as sc

In [70]:
train_df2 = train_df.drop(drop_cols, axis = 1)

In [71]:
test_df2 = test_df.drop(drop_cols, axis = 1)

In [72]:
for i in range(len(train_df2.columns)):
    if 'UGDS' in train_df2.columns[i]:
        if 'UGDS_WHITE' == train_df2.columns[i]:
            next
        else:
            x = sc.boxcox(np.array(train_df2[train_df2.columns[i]]),0.5)
            train_df2.loc[:,train_df2.columns[i]] = pd.Series(x, index=train_df2.index)
    if 'PCTFLOAN' in train_df2.columns[i]:
        x = sc.boxcox(np.array(train_df2[train_df2.columns[i]]),2)
        train_df2.loc[:,train_df2.columns[i]] = pd.Series(x, index=train_df2.index)
    if 'PPTUG_EF' in train_df2.columns[i]:
        x = sc.boxcox(np.array(train_df2[train_df2.columns[i]]),2)
        train_df2.loc[:,train_df2.columns[i]] = pd.Series(x, index=train_df2.index)

In [73]:
for i in range(len(test_df2.columns)):
    if 'UGDS' in test_df2.columns[i]:
        if 'UGDS_WHITE' == test_df2.columns[i]:
            next
        else:
            x = sc.boxcox(np.array(test_df2[test_df2.columns[i]]),0.5)
            test_df2.loc[:,test_df2.columns[i]] = pd.Series(x, index=test_df2.index)
    if 'PCTFLOAN' in test_df2.columns[i]:
        x = sc.boxcox(np.array(test_df2[test_df2.columns[i]]),2)
        test_df2.loc[:,test_df2.columns[i]] = pd.Series(x, index=test_df2.index)
    if 'PPTUG_EF' in test_df2.columns[i]:
        x = sc.boxcox(np.array(test_df2[test_df2.columns[i]]),2)
        test_df2.loc[:,test_df2.columns[i]] = pd.Series(x, index=test_df2.index)

In [74]:
train_df3 = train_df.drop(drop_cols, axis = 1)
test_df3 = test_df.drop(drop_cols, axis=1)

In [75]:
for i in range(len(train_df3.columns)):
    if 'UGDS' in train_df3.columns[i]:
        if 'UGDS_WHITE' == train_df3.columns[i]:
            next
        else:
            x = sc.boxcox(np.array(train_df3[train_df3.columns[i]]),0.5)
            train_df3.loc[:,train_df3.columns[i]] = pd.Series(x, index=train_df3.index)

In [76]:
for i in range(len(test_df3.columns)):
    if 'UGDS' in test_df3.columns[i]:
        if 'UGDS_WHITE' == test_df3.columns[i]:
            next
        else:
            x = sc.boxcox(np.array(test_df3[test_df3.columns[i]]),0.5)
            test_df3.loc[:,test_df3.columns[i]] = pd.Series(x, index=test_df3.index)


In [77]:
train_df = train_df.drop(drop_cols, axis = 1)
test_df = test_df.drop(drop_cols, axis=1)

In [78]:
folds = 100
shuffled = train_df.sample(frac=1)
print (len(shuffled))
idxs = np.arange(len(shuffled),dtype='i')/int(np.ceil(len(shuffled)/float(folds)))
#ind_folds = np.array([i%folds for i in range(len(shuffled))])
#ind_foldsp
print idxs

6243
[ 0  0  0 ... 99 99 99]


In [79]:
shuffled2 = train_df2.sample(frac=1)
shuffled3 = train_df3.sample(frac=1)

## Logistic Regression

In [29]:
fs = max(idxs)+1
accur = np.zeros(fs)
for f in range(fs):
    test = shuffled.iloc[np.where(idxs==i)]
    train = shuffled.iloc[np.where(idxs!=i)]
    
    predictions = [0.0 for i in range(63)]
        
    correct = 0.0
    count = 0.0
    for x in range(len(test)):
        if test.iloc[x]['Binary_25k'] == predictions[x]:
            correct =correct +1
            count = count + 1
        elif test.iloc[x]['Binary_25k'] == 'NaN':
            next
        else:
            count = count + 1
    accur[f] = correct/count

print (accur.mean())
print (accur.std(ddof=1)/np.sqrt(fs))

0.5561904761904761
0.0006349206349206344


In [30]:
fs = max(idxs)+1
accur = np.zeros(fs)
for f in range(fs):
    test = shuffled2.iloc[np.where(idxs==i)]
    train = shuffled2.iloc[np.where(idxs!=i)]
    
    predictions = [0.0 for i in range(63)]
        
    correct = 0.0
    count = 0.0
    for x in range(len(test)):
        if test.iloc[x]['Binary_25k'] == predictions[x]:
            correct =correct +1
            count = count + 1
        elif test.iloc[x]['Binary_25k'] == 'NaN':
            next
        else:
            count = count + 1
    accur[f] = correct/count

print (accur.mean())
print (accur.std(ddof=1)/np.sqrt(fs))

0.5079365079365078
1.1158161231197418e-17


In [31]:
fs = max(idxs)+1
accur = np.zeros(fs)
for f in range(fs):
    test = shuffled3.iloc[np.where(idxs==i)]
    train = shuffled3.iloc[np.where(idxs!=i)]
    
    predictions = [0.0 for i in range(63)]
        
    correct = 0.0
    count = 0.0
    for x in range(len(test)):
        if test.iloc[x]['Binary_25k'] == predictions[x]:
            correct =correct +1
            count = count + 1
        elif test.iloc[x]['Binary_25k'] == 'NaN':
            next
        else:
            count = count + 1
    accur[f] = correct/count

print (accur.mean())
print (accur.std(ddof=1)/np.sqrt(fs))

0.5873015873015873
0.0


In [80]:
total2 = 'PREDDEG + PCTPELL + GRAD_DEBT_MDN_SUPP + NET_COST'

In [83]:
smf.logit('Binary_25k ~ ' + total2 , data=train_df2).fit().summary()

Optimization terminated successfully.
         Current function value: 0.399775
         Iterations 7


<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:             Binary_25k   No. Observations:                 3900
Model:                          Logit   Df Residuals:                     3895
Method:                           MLE   Df Model:                            4
Date:                Tue, 01 May 2018   Pseudo R-squ.:                  0.3514
Time:                        20:47:30   Log-Likelihood:                -1559.1
converged:                       True   LL-Null:                       -2404.0
                                        LLR p-value:                     0.000
======================================================================================
                         coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------------
Intercept             -1.9345      0.225     -8.611      0.000      -2.375      -1.494
PREDDEG                0.8007      0.086      9.359      0.000       0.633       0.968
PCTPELL               -5.2507      0.290    -18.117      0.000      -5.819      -4.683
GRAD_DEBT_MDN_SUPP  5.824e-05   8.87e-06      6.564      0.000    4.08e-05    7.56e-05
NET_COST            4.539e-05   7.24e-06      6.271      0.000    3.12e-05    5.96e-05
======================================================================================
"""

In [95]:
new_labels

[0.0,
 0.0,
 1.0,
 0.0,
 0.0,
 1.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 1.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 1.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 1.0,
 0.0,
 0.0,
 0.0,
 1.0,
 0.0,
 0.0,
 0.0,
 1.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 1.0,
 0.0,
 0.0,
 0.0,
 0.0,
 1.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0]

In [96]:
fs = max(idxs)+1
accur = np.zeros(fs)
for f in range(fs):
    test = shuffled.iloc[np.where(idxs==i)]
    train = shuffled.iloc[np.where(idxs!=i)]
    
    lm = smf.logit('Binary_25k ~ ' + total2 , data=train_df).fit()
    predictions = lm.predict(test)
    
    new_labels = [0 for m in range(len(predictions))]
    
    for z in range(len(predictions)):
        if predictions.iloc[z] == 'NaN':
            new_labels[z] = 'NaN'
        elif predictions.iloc[z] > 0.5:
            new_labels[z] = 1.0
        else:
            new_labels[z] = 0.0
        
    correct = 0.0
    count = 0.0
    for x in range(len(test)):
        if new_labels[x] == 'NaN':
            next
        if test.iloc[x]['Binary_25k'] == new_labels[x]:
            correct =correct +1
            count = count + 1
        elif test.iloc[x]['Binary_25k'] == 'NaN':
            next
        else:
            count = count + 1
    accur[f] = correct/count

print (accur.mean())
print (accur.std(ddof=1)/np.sqrt(fs))

Optimization terminated successfully.
         Current function value: 0.399775
         Iterations 7
Optimization terminated successfully.
         Current function value: 0.399775
         Iterations 7
Optimization terminated successfully.
         Current function value: 0.399775
         Iterations 7
Optimization terminated successfully.
         Current function value: 0.399775
         Iterations 7
Optimization terminated successfully.
         Current function value: 0.399775
         Iterations 7
Optimization terminated successfully.
         Current function value: 0.399775
         Iterations 7
Optimization terminated successfully.
         Current function value: 0.399775
         Iterations 7
Optimization terminated successfully.
         Current function value: 0.399775
         Iterations 7
Optimization terminated successfully.
         Current function value: 0.399775
         Iterations 7
Optimization terminated successfully.
         Current function value: 0.399775
  

Optimization terminated successfully.
         Current function value: 0.399775
         Iterations 7
Optimization terminated successfully.
         Current function value: 0.399775
         Iterations 7
Optimization terminated successfully.
         Current function value: 0.399775
         Iterations 7
Optimization terminated successfully.
         Current function value: 0.399775
         Iterations 7
Optimization terminated successfully.
         Current function value: 0.399775
         Iterations 7
Optimization terminated successfully.
         Current function value: 0.399775
         Iterations 7
Optimization terminated successfully.
         Current function value: 0.399775
         Iterations 7
Optimization terminated successfully.
         Current function value: 0.399775
         Iterations 7
Optimization terminated successfully.
         Current function value: 0.399775
         Iterations 7
Optimization terminated successfully.
         Current function value: 0.399775
  

In [97]:
fs = max(idxs)+1
accur = np.zeros(fs)
for f in range(fs):
    test = shuffled2.iloc[np.where(idxs==i)]
    train = shuffled2.iloc[np.where(idxs!=i)]
    
    lm = smf.logit('Binary_25k ~ ' + total2 , data=train_df2).fit()
    predictions = lm.predict(test)
    
    new_labels = [0 for m in range(len(predictions))]
    
    for z in range(len(predictions)):
        if predictions.iloc[z] == 'NaN':
            new_labels[z] = 'NaN'
        elif predictions.iloc[z] > 0.5:
            new_labels[z] = 1.0
        else:
            new_labels[z] = 0.0
        
    correct = 0.0
    count = 0.0
    for x in range(len(test)):
        if new_labels[x] == 'NaN':
            next
        if test.iloc[x]['Binary_25k'] == new_labels[x]:
            correct =correct +1
            count = count + 1
        elif test.iloc[x]['Binary_25k'] == 'NaN':
            next
        else:
            count = count + 1
    accur[f] = correct/count

print (accur.mean())
print (accur.std(ddof=1)/np.sqrt(fs))

Optimization terminated successfully.
         Current function value: 0.399775
         Iterations 7
Optimization terminated successfully.
         Current function value: 0.399775
         Iterations 7
Optimization terminated successfully.
         Current function value: 0.399775
         Iterations 7
Optimization terminated successfully.
         Current function value: 0.399775
         Iterations 7
Optimization terminated successfully.
         Current function value: 0.399775
         Iterations 7
Optimization terminated successfully.
         Current function value: 0.399775
         Iterations 7
Optimization terminated successfully.
         Current function value: 0.399775
         Iterations 7
Optimization terminated successfully.
         Current function value: 0.399775
         Iterations 7
Optimization terminated successfully.
         Current function value: 0.399775
         Iterations 7
Optimization terminated successfully.
         Current function value: 0.399775
  

Optimization terminated successfully.
         Current function value: 0.399775
         Iterations 7
Optimization terminated successfully.
         Current function value: 0.399775
         Iterations 7
Optimization terminated successfully.
         Current function value: 0.399775
         Iterations 7
Optimization terminated successfully.
         Current function value: 0.399775
         Iterations 7
Optimization terminated successfully.
         Current function value: 0.399775
         Iterations 7
Optimization terminated successfully.
         Current function value: 0.399775
         Iterations 7
Optimization terminated successfully.
         Current function value: 0.399775
         Iterations 7
Optimization terminated successfully.
         Current function value: 0.399775
         Iterations 7
Optimization terminated successfully.
         Current function value: 0.399775
         Iterations 7
Optimization terminated successfully.
         Current function value: 0.399775
  

In [98]:
fs = max(idxs)+1
accur = np.zeros(fs)
for f in range(fs):
    test = shuffled3.iloc[np.where(idxs==i)]
    train = shuffled3.iloc[np.where(idxs!=i)]
    
    lm = smf.logit('Binary_25k ~ ' + total2 , data=train_df3).fit()
    predictions = lm.predict(test)
    
    new_labels = [0 for m in range(len(predictions))]
    
    for z in range(len(predictions)):
        if predictions.iloc[z] == 'NaN':
            new_labels[z] = 'NaN'
        elif predictions.iloc[z] > 0.5:
            new_labels[z] = 1.0
        else:
            new_labels[z] = 0.0
        
    correct = 0.0
    count = 0.0
    for x in range(len(test)):
        if new_labels[x] == 'NaN':
            next
        if test.iloc[x]['Binary_25k'] == new_labels[x]:
            correct =correct +1
            count = count + 1
        elif test.iloc[x]['Binary_25k'] == 'NaN':
            next
        else:
            count = count + 1
    accur[f] = correct/count

print (accur.mean())
print (accur.std(ddof=1)/np.sqrt(fs))

Optimization terminated successfully.
         Current function value: 0.399775
         Iterations 7
Optimization terminated successfully.
         Current function value: 0.399775
         Iterations 7
Optimization terminated successfully.
         Current function value: 0.399775
         Iterations 7
Optimization terminated successfully.
         Current function value: 0.399775
         Iterations 7
Optimization terminated successfully.
         Current function value: 0.399775
         Iterations 7
Optimization terminated successfully.
         Current function value: 0.399775
         Iterations 7
Optimization terminated successfully.
         Current function value: 0.399775
         Iterations 7
Optimization terminated successfully.
         Current function value: 0.399775
         Iterations 7
Optimization terminated successfully.
         Current function value: 0.399775
         Iterations 7
Optimization terminated successfully.
         Current function value: 0.399775
  

Optimization terminated successfully.
         Current function value: 0.399775
         Iterations 7
Optimization terminated successfully.
         Current function value: 0.399775
         Iterations 7
Optimization terminated successfully.
         Current function value: 0.399775
         Iterations 7
Optimization terminated successfully.
         Current function value: 0.399775
         Iterations 7
Optimization terminated successfully.
         Current function value: 0.399775
         Iterations 7
Optimization terminated successfully.
         Current function value: 0.399775
         Iterations 7
Optimization terminated successfully.
         Current function value: 0.399775
         Iterations 7
Optimization terminated successfully.
         Current function value: 0.399775
         Iterations 7
Optimization terminated successfully.
         Current function value: 0.399775
         Iterations 7
Optimization terminated successfully.
         Current function value: 0.399775
  

In [99]:
import itertools

In [100]:
singlefeatures=list(set(train_df2.columns)-{'RPY_3YR_RT_SUPP'} - {'Binary_25k'})
print singlefeatures

['CONTROL', 'UGDS_AIAN', 'PPTUG_EF', 'GRAD_DEBT_MDN_SUPP', 'PREDDEG', 'UG25abv', 'LOCALE', 'UGDS_NRA', 'UGDS_ASIAN', 'PCTPELL', 'NET_COST', 'UGDS_2MOR', 'UGDS_NHPI', 'UGDS_BLACK', 'UGDS', 'DISTANCEONLY', 'UGDS_WHITE', 'PCTFLOAN', 'UGDS_HISP']


In [101]:
 smf.logit('Binary_25k~ ' + '+'.join(singlefeatures), data=train_df).fit().summary()

Optimization terminated successfully.
         Current function value: 0.340499
         Iterations 8


<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:             Binary_25k   No. Observations:                 3896
Model:                          Logit   Df Residuals:                     3876
Method:                           MLE   Df Model:                           19
Date:                Tue, 01 May 2018   Pseudo R-squ.:                  0.4475
Time:                        20:58:26   Log-Likelihood:                -1326.6
converged:                       True   LL-Null:                       -2400.9
                                        LLR p-value:                     0.000
======================================================================================
                         coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------------
Intercept             -1.5983      0.639     -2.501      0.012      -2.851      -0.346
CONTROL               -1.1036      0.107    -10.287      0.000      -1.314      -0.893
UGDS_AIAN             -9.9573      3.849     -2.587      0.010     -17.502      -2.413
PPTUG_EF              -1.9268      0.277     -6.954      0.000      -2.470      -1.384
GRAD_DEBT_MDN_SUPP  2.313e-05    1.1e-05      2.103      0.035    1.57e-06    4.47e-05
PREDDEG                0.8133      0.114      7.143      0.000       0.590       1.036
UG25abv                4.3175      0.343     12.580      0.000       3.645       4.990
LOCALE                -0.0020      0.006     -0.334      0.738      -0.014       0.010
UGDS_NRA              -3.7994      1.596     -2.381      0.017      -6.927      -0.672
UGDS_ASIAN             8.3411      1.271      6.561      0.000       5.849      10.833
PCTPELL               -9.9747      0.567    -17.591      0.000     -11.086      -8.863
NET_COST            4.038e-05   1.09e-05      3.713      0.000    1.91e-05    6.17e-05
UGDS_2MOR             -4.1091      2.101     -1.956      0.050      -8.226       0.008
UGDS_NHPI              3.2040      2.909      1.101      0.271      -2.497       8.905
UGDS_BLACK            -0.1907      0.521     -0.366      0.714      -1.212       0.831
UGDS                2.951e-05   1.03e-05      2.860      0.004    9.29e-06    4.97e-05
DISTANCEONLY           0.6921      0.752      0.920      0.357      -0.782       2.166
UGDS_WHITE            -0.0895      0.480     -0.186      0.852      -1.030       0.851
PCTFLOAN               5.9576      0.453     13.152      0.000       5.070       6.845
UGDS_HISP              0.5148      0.591      0.871      0.384      -0.643       1.673
======================================================================================
"""

In [102]:
fs = max(idxs)+1
accur = np.zeros(fs)
for f in range(fs):
    test = shuffled.iloc[np.where(idxs==i)]
    train = shuffled.iloc[np.where(idxs!=i)]
    
    lm = smf.logit('Binary_25k ~ ' +'+'.join(singlefeatures) , data=train_df).fit()
    predictions = lm.predict(test)
    
    new_labels = [0 for m in range(len(predictions))]
    
    for z in range(len(predictions)):
        if predictions.iloc[z] == 'NaN':
            new_labels[z] = 'NaN'
        elif predictions.iloc[z] > 0.5:
            new_labels[z] = 1.0
        else:
            new_labels[z] = 0.0
        
    correct = 0.0
    count = 0.0
    for x in range(len(test)):
        if new_labels[x] == 'NaN':
            next
        if test.iloc[x]['Binary_25k'] == new_labels[x]:
            correct =correct +1
            count = count + 1
        elif test.iloc[x]['Binary_25k'] == 'NaN':
            next
        else:
            count = count + 1
    accur[f] = correct/count

print (accur.mean())
print (accur.std(ddof=1)/np.sqrt(fs))

Optimization terminated successfully.
         Current function value: 0.340499
         Iterations 8
Optimization terminated successfully.
         Current function value: 0.340499
         Iterations 8
Optimization terminated successfully.
         Current function value: 0.340499
         Iterations 8
Optimization terminated successfully.
         Current function value: 0.340499
         Iterations 8
Optimization terminated successfully.
         Current function value: 0.340499
         Iterations 8
Optimization terminated successfully.
         Current function value: 0.340499
         Iterations 8
Optimization terminated successfully.
         Current function value: 0.340499
         Iterations 8
Optimization terminated successfully.
         Current function value: 0.340499
         Iterations 8
Optimization terminated successfully.
         Current function value: 0.340499
         Iterations 8
Optimization terminated successfully.
         Current function value: 0.340499
  

Optimization terminated successfully.
         Current function value: 0.340499
         Iterations 8
Optimization terminated successfully.
         Current function value: 0.340499
         Iterations 8
Optimization terminated successfully.
         Current function value: 0.340499
         Iterations 8
Optimization terminated successfully.
         Current function value: 0.340499
         Iterations 8
Optimization terminated successfully.
         Current function value: 0.340499
         Iterations 8
Optimization terminated successfully.
         Current function value: 0.340499
         Iterations 8
Optimization terminated successfully.
         Current function value: 0.340499
         Iterations 8
Optimization terminated successfully.
         Current function value: 0.340499
         Iterations 8
Optimization terminated successfully.
         Current function value: 0.340499
         Iterations 8
Optimization terminated successfully.
         Current function value: 0.340499
  

## Test SET

In [136]:
predictions = [np.mean(train_df2['RPY_3YR_RT_SUPP']) for i in range(test_df.shape[0])]
predictions1 = [0 for i in range(test_df.shape[0])]

In [135]:
((test_df2['RPY_3YR_RT_SUPP'] - predictions)**2).mean()

0.04172288540698032

In [137]:
((test_df2['Binary_25k'] - predictions1)**2).mean()

0.30658783783783783

In [22]:
import itertools

In [100]:
singlefeatures=list(set(train_df2.columns)-{'RPY_3YR_RT_SUPP'} - {'Binary_25k'})
print singlefeatures

['CONTROL', 'UGDS_AIAN', 'PPTUG_EF', 'GRAD_DEBT_MDN_SUPP', 'PREDDEG', 'UG25abv', 'LOCALE', 'UGDS_NRA', 'UGDS_ASIAN', 'PCTPELL', 'NET_COST', 'UGDS_2MOR', 'UGDS_NHPI', 'UGDS_BLACK', 'UGDS', 'DISTANCEONLY', 'UGDS_WHITE', 'PCTFLOAN', 'UGDS_HISP']


In [101]:
smf.ols('RPY_3YR_RT_SUPP~ ' + '+'.join(singlefeatures), data=train_df2).fit().summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:        RPY_3YR_RT_SUPP   R-squared:                       0.693
Model:                            OLS   Adj. R-squared:                  0.691
Method:                 Least Squares   F-statistic:                     509.0
Date:                Sun, 29 Apr 2018   Prob (F-statistic):               0.00
Time:                        13:40:18   Log-Likelihood:                 3268.1
No. Observations:                4309   AIC:                            -6496.
Df Residuals:                    4289   BIC:                            -6369.
Df Model:                          19                                         
Covariance Type:            nonrobust                                         
======================================================================================
                         coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------
Intercept              1.0268      0.057     17.885      0.000       0.914       1.139
CONTROL               -0.0364      0.004     -9.863      0.000      -0.044      -0.029
UGDS_AIAN             -0.0982      0.013     -7.517      0.000      -0.124      -0.073
PPTUG_EF              -0.0391      0.026     -1.479      0.139      -0.091       0.013
GRAD_DEBT_MDN_SUPP -3.285e-06   3.64e-07     -9.012      0.000      -4e-06   -2.57e-06
PREDDEG                0.0499      0.004     13.001      0.000       0.042       0.057
UG25abv               -0.1223      0.011    -11.006      0.000      -0.144      -0.100
LOCALE                -0.0005      0.000     -2.065      0.039      -0.001   -2.36e-05
UGDS_NRA               0.1098      0.013      8.536      0.000       0.085       0.135
UGDS_ASIAN             0.1417      0.010     14.019      0.000       0.122       0.162
PCTPELL               -0.4402      0.016    -28.139      0.000      -0.471      -0.410
NET_COST            1.227e-06   3.45e-07      3.560      0.000    5.51e-07     1.9e-06
UGDS_2MOR              0.0228      0.011      2.151      0.032       0.002       0.044
UGDS_NHPI             -0.1069      0.020     -5.466      0.000      -0.145      -0.069
UGDS_BLACK            -0.0659      0.007    -10.056      0.000      -0.079      -0.053
UGDS                  -0.0003   3.77e-05     -7.806      0.000      -0.000      -0.000
DISTANCEONLY           0.0619      0.029      2.129      0.033       0.005       0.119
UGDS_WHITE             0.1249      0.013      9.694      0.000       0.100       0.150
PCTFLOAN               0.4573      0.023     19.731      0.000       0.412       0.503
UGDS_HISP              0.0149      0.007      2.210      0.027       0.002       0.028
==============================================================================
Omnibus:                       78.162   Durbin-Watson:                   2.004
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              150.076
Skew:                           0.084   Prob(JB):                     2.58e-33
Kurtosis:                       3.899   Cond. No.                     8.95e+05
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 8.95e+05. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [116]:
response = 'RPY_3YR_RT_SUPP'
lm = smf.ols('RPY_3YR_RT_SUPP~ ' + '+'.join(singlefeatures), data=train_df2).fit()

In [105]:
((test_df2[response] - lm.predict(test_df2))**2).mean()

0.013175717328176085

In [117]:
def fitmodel(S,data,response):
    return smf.ols(response+' ~ '+('+'.join(S) if len(S)>0 else '1'), data=data).fit()
def getaic(lm):
    return lm.aic
mycriterion = getaic

In [118]:
Sfwd = set()
features = set(singlefeatures)
while len(Sfwd)<len(features):
    f = min(features - Sfwd, key = lambda f: fitmodel(Sfwd.union({f}),train_df2, response).aic)
    after = fitmodel(Sfwd.union({f}),train_df2, response).aic
    before = fitmodel(Sfwd,train_df2, response).aic
    if after < before:
        Sfwd = Sfwd.union({f})
    else:
        break
print Sfwd
print len(Sfwd)

set(['UGDS_AIAN', 'CONTROL', 'DISTANCEONLY', 'PREDDEG', 'UG25abv', 'LOCALE', 'UGDS_ASIAN', 'PCTPELL', 'PCTFLOAN', 'UGDS_WHITE', 'UGDS_NHPI', 'UGDS_BLACK', 'UGDS', 'UGDS_NRA', 'UGDS_HISP'])
15


In [119]:
Sbwd = set(features)
while len(Sbwd)>0:
    f = min(Sbwd, key = lambda f: fitmodel(Sbwd-{f},train_df2, response).aic)
    after = fitmodel(Sbwd-{f},train_df2, response).aic
    before = fitmodel(Sbwd,train_df2, response).aic
    if after < before:
        Sbwd = Sbwd-{f}
    else:
        break
print Sbwd
print len(Sbwd)

set(['UGDS_AIAN', 'CONTROL', 'PREDDEG', 'UGDS_HISP', 'UG25abv', 'LOCALE', 'UGDS_ASIAN', 'PCTPELL', 'UGDS_WHITE', 'UGDS_NHPI', 'UGDS_BLACK', 'UGDS', 'DISTANCEONLY', 'PCTFLOAN', 'UGDS_NRA'])
15


In [122]:
lm1 = smf.ols('RPY_3YR_RT_SUPP~ ' + '+'.join(Sfwd), data=train_df2).fit()
lm2 = smf.ols('RPY_3YR_RT_SUPP~ ' + '+'.join(Sbwd), data=train_df2).fit()

In [121]:
((test_df2[response] - lm1.predict(test_df2))**2).mean()

0.014270258125751554

In [69]:
def CV_MSE(response, features, data, idxs):
    fs = max(idxs)+1
    mses = np.zeros(fs)
    for f in range(fs):
        mses[f] = ((data[idxs==f][response]
            -
            fitmodel(features,data[idxs!=f],response).predict(data[idxs==f]))**2).mean()
    return mses.mean(), mses.std(ddof=1)/np.sqrt(fs)

In [70]:
CV_MSE('RPY_3YR_RT_SUPP', singlefeatures, df3, idxs)

(0.013623279731487734, 0.0008123126250176889)

In [106]:
new_keeps = []
under_perc = []
for i in range(len(singlefeatures)):
    if 'UGDS_' in keep_variables[i]:
        under_perc.append(keep_variables[i])
    else:
        new_keeps.append(keep_variables[i])

In [107]:
singlefeatures1=list(set(new_keeps)-{'DISTANCEONLY'}-{'GRAD_DEBT_MDN10YR_SUPP'})

In [108]:
interactions1 = ['%s:%s'%v for v in itertools.combinations(singlefeatures1,2)]
print interactions1[:10]

[u'CONTROL:PPTUG_EF', u'CONTROL:GRAD_DEBT_MDN_SUPP', u'CONTROL:PREDDEG', u'CONTROL:UG25abv', u'CONTROL:LOCALE', u'CONTROL:PCTPELL', u'CONTROL:UGDS', u'CONTROL:PCTFLOAN', u'PPTUG_EF:GRAD_DEBT_MDN_SUPP', u'PPTUG_EF:PREDDEG']


In [109]:
smf.logit('Binary_25k~ ' + '+'.join(interactions1), data=train_df2).fit().summary()

Optimization terminated successfully.
         Current function value: 0.304654
         Iterations 8


<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:             Binary_25k   No. Observations:                 4050
Model:                          Logit   Df Residuals:                     4013
Method:                           MLE   Df Model:                           36
Date:                Sun, 29 Apr 2018   Pseudo R-squ.:                  0.5142
Time:                        13:44:05   Log-Likelihood:                -1233.8
converged:                       True   LL-Null:                       -2539.6
                                        LLR p-value:                     0.000
===============================================================================================
                                  coef    std err          z      P>|z|      [0.025      0.975]
-----------------------------------------------------------------------------------------------
Intercept                       2.4674      0.386      6.396      0.000       1.711       3.224
CONTROL:PPTUG_EF                0.3587      0.791      0.453      0.650      -1.192       1.909
CONTROL:GRAD_DEBT_MDN_SUPP  -6.727e-05   1.65e-05     -4.077      0.000   -9.96e-05   -3.49e-05
CONTROL:PREDDEG                 0.6050      0.163      3.701      0.000       0.285       0.925
CONTROL:UG25abv                -1.0950      0.429     -2.551      0.011      -1.936      -0.254
CONTROL:LOCALE                 -0.0020      0.010     -0.209      0.834      -0.021       0.017
CONTROL:PCTPELL                -1.2067      0.528     -2.285      0.022      -2.242      -0.172
CONTROL:UGDS                    0.0210      0.002      8.761      0.000       0.016       0.026
CONTROL:PCTFLOAN                3.0059      0.807      3.723      0.000       1.423       4.588
PPTUG_EF:GRAD_DEBT_MDN_SUPP    -0.0002   8.23e-05     -2.206      0.027      -0.000   -2.02e-05
PPTUG_EF:PREDDEG                3.8956      0.921      4.228      0.000       2.090       5.702
PPTUG_EF:UG25abv                1.5310      3.551      0.431      0.666      -5.430       8.492
PPTUG_EF:LOCALE                -0.4231      0.070     -6.070      0.000      -0.560      -0.286
PPTUG_EF:PCTPELL                5.0632      3.372      1.502      0.133      -1.546      11.672
PPTUG_EF:UGDS                  -0.0347      0.013     -2.660      0.008      -0.060      -0.009
PPTUG_EF:PCTFLOAN              -4.5207      4.444     -1.017      0.309     -13.231       4.189
GRAD_DEBT_MDN_SUPP:PREDDEG    4.53e-06   1.02e-05      0.444      0.657   -1.55e-05    2.45e-05
GRAD_DEBT_MDN_SUPP:UG25abv      0.0002   5.61e-05      4.118      0.000       0.000       0.000
GRAD_DEBT_MDN_SUPP:LOCALE    2.511e-06   1.16e-06      2.166      0.030    2.39e-07    4.78e-06
GRAD_DEBT_MDN_SUPP:PCTPELL     -0.0001   7.01e-05     -1.706      0.088      -0.000    1.78e-05
GRAD_DEBT_MDN_SUPP:UGDS     -1.954e-06   2.77e-07     -7.043      0.000    -2.5e-06   -1.41e-06
GRAD_DEBT_MDN_SUPP:PCTFLOAN    -0.0005   9.95e-05     -4.671      0.000      -0.001      -0.000
PREDDEG:UG25abv                 1.2616      0.536      2.356      0.018       0.212       2.311
PREDDEG:LOCALE                 -0.0373      0.011     -3.260      0.001      -0.060      -0.015
PREDDEG:PCTPELL                -0.3743      0.719     -0.521      0.602      -1.783       1.034
PREDDEG:UGDS                    0.0238      0.003      9.068      0.000       0.019       0.029
PREDDEG:PCTFLOAN                1.4203      0.926      1.534      0.125      -0.395       3.235
UG25abv:LOCALE                  0.0088      0.035      0.251      0.802      -0.060       0.078
UG25abv:PCTPELL                 3.4521      1.612      2.141      0.032       0.292       6.612
UG25abv:UGDS                   -0.0330      0.008     -4.168      0.000      -0.048      -0.017
UG25abv:PCTFLOAN                2.7113      2.667      1.017      0.309  

In [124]:
response = 'Binary_25k'
logit = smf.logit('Binary_25k~ ' + '+'.join(interactions1), data=train_df2).fit()

Optimization terminated successfully.
         Current function value: 0.304654
         Iterations 8


In [112]:
((test_df2[response] - logit.predict(test_df2))**2).mean()

0.09583043116305355

In [125]:
def fitmodel_logit(S,data,response):
    return smf.logit(response+' ~ '+('+'.join(S) if len(S)>0 else '1'), data=data).fit()

In [126]:
Sfwd = set()
features = set(interactions1)
while len(Sfwd)<len(features):
    f = min(features - Sfwd, key = lambda f: fitmodel_logit(Sfwd.union({f}),train_df2, response).aic)
    after = fitmodel_logit(Sfwd.union({f}),train_df2, response).aic
    before = fitmodel_logit(Sfwd,train_df2, response).aic
    if after < before:
        Sfwd = Sfwd.union({f})
    else:
        break
print Sfwd
print len(Sfwd)

Optimization terminated successfully.
         Current function value: 0.592103
         Iterations 5
Optimization terminated successfully.
         Current function value: 0.571644
         Iterations 5
Optimization terminated successfully.
         Current function value: 0.576254
         Iterations 6
Optimization terminated successfully.
         Current function value: 0.626956
         Iterations 5
Optimization terminated successfully.
         Current function value: 0.551396
         Iterations 6
Optimization terminated successfully.
         Current function value: 0.560809
         Iterations 5
Optimization terminated successfully.
         Current function value: 0.594767
         Iterations 5
Optimization terminated successfully.
         Current function value: 0.559046
         Iterations 6
Optimization terminated successfully.
         Current function value: 0.592265
         Iterations 5
Optimization terminated successfully.
         Current function value: 0.579124
  

Optimization terminated successfully.
         Current function value: 0.431775
         Iterations 7
Optimization terminated successfully.
         Current function value: 0.429238
         Iterations 7
Optimization terminated successfully.
         Current function value: 0.430798
         Iterations 7
Optimization terminated successfully.
         Current function value: 0.430908
         Iterations 7
Optimization terminated successfully.
         Current function value: 0.431938
         Iterations 7
Optimization terminated successfully.
         Current function value: 0.431797
         Iterations 7
Optimization terminated successfully.
         Current function value: 0.428250
         Iterations 7
Optimization terminated successfully.
         Current function value: 0.426631
         Iterations 7
Optimization terminated successfully.
         Current function value: 0.431130
         Iterations 7
Optimization terminated successfully.
         Current function value: 0.431951
  

Optimization terminated successfully.
         Current function value: 0.407100
         Iterations 7
Optimization terminated successfully.
         Current function value: 0.393919
         Iterations 7
Optimization terminated successfully.
         Current function value: 0.405964
         Iterations 7
Optimization terminated successfully.
         Current function value: 0.404488
         Iterations 7
Optimization terminated successfully.
         Current function value: 0.405083
         Iterations 7
Optimization terminated successfully.
         Current function value: 0.405575
         Iterations 7
Optimization terminated successfully.
         Current function value: 0.406779
         Iterations 7
Optimization terminated successfully.
         Current function value: 0.393919
         Iterations 7
Optimization terminated successfully.
         Current function value: 0.407692
         Iterations 7
Optimization terminated successfully.
         Current function value: 0.393918
  

Optimization terminated successfully.
         Current function value: 0.362416
         Iterations 7
Optimization terminated successfully.
         Current function value: 0.365537
         Iterations 7
Optimization terminated successfully.
         Current function value: 0.365617
         Iterations 7
Optimization terminated successfully.
         Current function value: 0.365638
         Iterations 7
Optimization terminated successfully.
         Current function value: 0.361209
         Iterations 7
Optimization terminated successfully.
         Current function value: 0.364192
         Iterations 7
Optimization terminated successfully.
         Current function value: 0.365772
         Iterations 7
Optimization terminated successfully.
         Current function value: 0.365615
         Iterations 7
Optimization terminated successfully.
         Current function value: 0.359735
         Iterations 7
Optimization terminated successfully.
         Current function value: 0.364043
  

Optimization terminated successfully.
         Current function value: 0.342259
         Iterations 7
Optimization terminated successfully.
         Current function value: 0.342047
         Iterations 8
Optimization terminated successfully.
         Current function value: 0.343511
         Iterations 7
Optimization terminated successfully.
         Current function value: 0.344177
         Iterations 7
Optimization terminated successfully.
         Current function value: 0.339349
         Iterations 7
Optimization terminated successfully.
         Current function value: 0.344091
         Iterations 8
Optimization terminated successfully.
         Current function value: 0.344503
         Iterations 7
Optimization terminated successfully.
         Current function value: 0.343994
         Iterations 8
Optimization terminated successfully.
         Current function value: 0.343810
         Iterations 7
Optimization terminated successfully.
         Current function value: 0.343417
  

Optimization terminated successfully.
         Current function value: 0.323665
         Iterations 7
Optimization terminated successfully.
         Current function value: 0.323473
         Iterations 7
Optimization terminated successfully.
         Current function value: 0.323649
         Iterations 7
Optimization terminated successfully.
         Current function value: 0.323249
         Iterations 7
Optimization terminated successfully.
         Current function value: 0.323087
         Iterations 7
Optimization terminated successfully.
         Current function value: 0.323572
         Iterations 7
Optimization terminated successfully.
         Current function value: 0.323552
         Iterations 7
Optimization terminated successfully.
         Current function value: 0.323248
         Iterations 7
Optimization terminated successfully.
         Current function value: 0.323150
         Iterations 7
Optimization terminated successfully.
         Current function value: 0.323791
  

Optimization terminated successfully.
         Current function value: 0.316052
         Iterations 8
Optimization terminated successfully.
         Current function value: 0.316315
         Iterations 7
Optimization terminated successfully.
         Current function value: 0.316248
         Iterations 7
Optimization terminated successfully.
         Current function value: 0.315434
         Iterations 7
Optimization terminated successfully.
         Current function value: 0.316293
         Iterations 7
Optimization terminated successfully.
         Current function value: 0.316256
         Iterations 7
Optimization terminated successfully.
         Current function value: 0.316155
         Iterations 7
Optimization terminated successfully.
         Current function value: 0.315908
         Iterations 7
Optimization terminated successfully.
         Current function value: 0.313890
         Iterations 7
Optimization terminated successfully.
         Current function value: 0.316163
  

Optimization terminated successfully.
         Current function value: 0.311157
         Iterations 8
Optimization terminated successfully.
         Current function value: 0.310411
         Iterations 8
Optimization terminated successfully.
         Current function value: 0.311167
         Iterations 8
Optimization terminated successfully.
         Current function value: 0.310372
         Iterations 8
Optimization terminated successfully.
         Current function value: 0.310386
         Iterations 8
Optimization terminated successfully.
         Current function value: 0.310325
         Iterations 8
Optimization terminated successfully.
         Current function value: 0.310402
         Iterations 8
Optimization terminated successfully.
         Current function value: 0.309999
         Iterations 8
Optimization terminated successfully.
         Current function value: 0.310258
         Iterations 8
Optimization terminated successfully.
         Current function value: 0.310379
  

In [127]:
Sbwd = set(features)
while len(Sbwd)>0:
    f = min(Sbwd, key = lambda f: fitmodel_logit(Sbwd-{f},train_df2, response).aic)
    after = fitmodel_logit(Sbwd-{f},train_df2, response).aic
    before = fitmodel_logit(Sbwd,train_df2, response).aic
    if after < before:
        Sbwd = Sbwd-{f}
    else:
        break
print Sbwd
print len(Sbwd)

Optimization terminated successfully.
         Current function value: 0.305234
         Iterations 8
Optimization terminated successfully.
         Current function value: 0.305965
         Iterations 8
Optimization terminated successfully.
         Current function value: 0.304677
         Iterations 8
Optimization terminated successfully.
         Current function value: 0.305012
         Iterations 8
Optimization terminated successfully.
         Current function value: 0.306849
         Iterations 8
Optimization terminated successfully.
         Current function value: 0.315179
         Iterations 8
Optimization terminated successfully.
         Current function value: 0.304659
         Iterations 8
Optimization terminated successfully.
         Current function value: 0.305219
         Iterations 8
Optimization terminated successfully.
         Current function value: 0.314589
         Iterations 8
Optimization terminated successfully.
         Current function value: 0.305459
  

Optimization terminated successfully.
         Current function value: 0.307571
         Iterations 8
Optimization terminated successfully.
         Current function value: 0.304700
         Iterations 8
Optimization terminated successfully.
         Current function value: 0.306804
         Iterations 8
Optimization terminated successfully.
         Current function value: 0.306650
         Iterations 8
Optimization terminated successfully.
         Current function value: 0.306848
         Iterations 8
Optimization terminated successfully.
         Current function value: 0.306761
         Iterations 8
Optimization terminated successfully.
         Current function value: 0.304692
         Iterations 8
Optimization terminated successfully.
         Current function value: 0.306358
         Iterations 8
Optimization terminated successfully.
         Current function value: 0.308227
         Iterations 8
Optimization terminated successfully.
         Current function value: 0.305267
  

Optimization terminated successfully.
         Current function value: 0.310771
         Iterations 8
Optimization terminated successfully.
         Current function value: 0.305539
         Iterations 8
Optimization terminated successfully.
         Current function value: 0.304896
         Iterations 8
Optimization terminated successfully.
         Current function value: 0.306835
         Iterations 8
Optimization terminated successfully.
         Current function value: 0.305420
         Iterations 8
Optimization terminated successfully.
         Current function value: 0.304978
         Iterations 8
Optimization terminated successfully.
         Current function value: 0.304977
         Iterations 8
Optimization terminated successfully.
         Current function value: 0.304770
         Iterations 8
Optimization terminated successfully.
         Current function value: 0.305208
         Iterations 8
Optimization terminated successfully.
         Current function value: 0.311659
  

Optimization terminated successfully.
         Current function value: 0.306045
         Iterations 8
Optimization terminated successfully.
         Current function value: 0.316084
         Iterations 8
Optimization terminated successfully.
         Current function value: 0.306413
         Iterations 8
Optimization terminated successfully.
         Current function value: 0.312154
         Iterations 8
Optimization terminated successfully.
         Current function value: 0.308474
         Iterations 8
Optimization terminated successfully.
         Current function value: 0.308469
         Iterations 8
Optimization terminated successfully.
         Current function value: 0.311626
         Iterations 8
Optimization terminated successfully.
         Current function value: 0.305845
         Iterations 8
Optimization terminated successfully.
         Current function value: 0.307283
         Iterations 8
Optimization terminated successfully.
         Current function value: 0.307479
  

In [128]:
logit1 = smf.logit('Binary_25k~ ' + '+'.join(Sfwd), data=train_df2).fit()
logit2 = smf.logit('Binary_25k~ ' + '+'.join(Sbwd), data=train_df2).fit()

Optimization terminated successfully.
         Current function value: 0.306909
         Iterations 8
Optimization terminated successfully.
         Current function value: 0.305052
         Iterations 8


In [129]:
((test_df2[response] - logit1.predict(test_df2))**2).mean()

0.0963734979017415

In [130]:
((test_df2[response] - logit2.predict(test_df2))**2).mean()

0.09649689776314016

In [142]:
logit4 = smf.logit('Binary_25k~ ' + '+'.join(interactions1), data=train_df3).fit()

Optimization terminated successfully.
         Current function value: 0.300947
         Iterations 8


In [143]:
((test_df3[response] - logit4.predict(test_df3))**2).mean()

0.09380662645276365

In [76]:
def CV_MSE_logit(response, features, data, idxs):
    fs = max(idxs)+1
    mses = np.zeros(fs)
    for f in range(fs):
        mses[f] = ((data[idxs==f][response]
            -
            fitmodel_logit(features,data[idxs!=f],response).predict(data[idxs==f]))**2).mean()
    return mses.mean(), mses.std(ddof=1)/np.sqrt(fs)

In [77]:
CV_MSE_logit('Binary_25k', singlefeatures, df3, idxs)

Optimization terminated successfully.
         Current function value: 0.555327
         Iterations 6
Optimization terminated successfully.
         Current function value: 0.555331
         Iterations 6
Optimization terminated successfully.
         Current function value: 0.555573
         Iterations 6
Optimization terminated successfully.
         Current function value: 0.556894
         Iterations 5
Optimization terminated successfully.
         Current function value: 0.554375
         Iterations 6
Optimization terminated successfully.
         Current function value: 0.554797
         Iterations 6
Optimization terminated successfully.
         Current function value: 0.554110
         Iterations 6
Optimization terminated successfully.
         Current function value: 0.556219
         Iterations 6
Optimization terminated successfully.
         Current function value: 0.554478
         Iterations 6
Optimization terminated successfully.
         Current function value: 0.555386
  

Optimization terminated successfully.
         Current function value: 0.555214
         Iterations 6
Optimization terminated successfully.
         Current function value: 0.555761
         Iterations 6
Optimization terminated successfully.
         Current function value: 0.556678
         Iterations 5
Optimization terminated successfully.
         Current function value: 0.555698
         Iterations 6
Optimization terminated successfully.
         Current function value: 0.557268
         Iterations 6
Optimization terminated successfully.
         Current function value: 0.555030
         Iterations 6
Optimization terminated successfully.
         Current function value: 0.556319
         Iterations 6
Optimization terminated successfully.
         Current function value: 0.554594
         Iterations 6
Optimization terminated successfully.
         Current function value: 0.555411
         Iterations 6
Optimization terminated successfully.
         Current function value: 0.555820
  

(0.014064675624213105, 0.0008588643209417316)